<a href="https://colab.research.google.com/github/Ankur3107/Machine-Learning-Notes/blob/master/Genre_Classification%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q transformers

     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
     |████████████████████████████████| 2.9MB 55.4MB/s 


In [5]:
import os
os.environ['KAGGLE_USERNAME'] = "XXXXXX"
os.environ['KAGGLE_KEY'] = "XXXXX"

In [7]:
!kaggle datasets download -d jrobischon/wikipedia-movie-plots

 60% 18.0M/29.9M [00:00<00:00, 31.4MB/s]
100% 29.9M/29.9M [00:00<00:00, 75.6MB/s]


In [12]:
!unzip wikipedia-movie-plots

Archive:  wikipedia-movie-plots.zip
  inflating: wiki_movie_plots_deduped.csv  


In [9]:
import pandas as pd
import json
from sklearn.preprocessing import MultiLabelBinarizer
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras
from sklearn.model_selection import train_test_split
import warnings
import ast
warnings.filterwarnings('ignore')

In [87]:
@dataclass
class Config:
    MAX_LEN = 512
    BATCH_SIZE = 4
    LR = 1e-5
    PRETRAINED_MODEL = "aloxatel/bert-base-mnli"  # huggingface bert model
    from_pt = True


config = Config()

In [11]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

In [33]:
df = pd.read_csv("wiki_movie_plots_deduped.csv")
df['id'] = list(range(df.shape[0]))
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,id
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",0
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",1
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",2
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,3
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,4


In [50]:
df['Genre_list'] = df['Genre'].apply(lambda x: [i.strip() for i in x.split(",")])

In [52]:
df_explode = df.explode("Genre_list")
len(df_explode.Genre_list.unique())

1449

In [53]:
shortlisted_genres = df_explode["Genre_list"].value_counts().reset_index(name="count").query("count > 100")["index"].tolist()

In [54]:
shortlisted_genres.remove("unknown")

In [55]:
len(shortlisted_genres)

30

In [81]:
genres_dict = dict(zip(shortlisted_genres, [str(i) for i in range(len(shortlisted_genres))]))

In [82]:
df_filtered = df_explode[df_explode['Genre_list'].isin(shortlisted_genres)]
df_filtered['Genre_target'] = df_filtered['Genre_list'].map(genres_dict)
df_filtered.shape

(27635, 11)

In [83]:
df_filtered['Genre_target'] = df_filtered[['id','Plot','Genre_target']].groupby(['id','Plot'])['Genre_target'].transform(lambda x: ','.join(x))

In [85]:
df_filtered['Genre_target'] = df_filtered['Genre_target'].apply(lambda x: [int(i) for i in x.split(",")])

In [90]:
tokenizer = AutoTokenizer.from_pretrained(config.PRETRAINED_MODEL)

X_train = regular_encode(df_filtered.Plot.values.tolist(), tokenizer, maxlen=config.MAX_LEN)
y_train = df_filtered.Genre_target.values

In [91]:
multi_label_binarizer = MultiLabelBinarizer()
y_train = multi_label_binarizer.fit_transform(y_train)
y_train.shape, X_train.shape

((27635, 30), (27635, 512))

In [92]:
multi_label_binarizer.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [93]:
def build_model():
    input_layer = tf.keras.layers.Input((config.MAX_LEN,), dtype=tf.int32)
    bert_model = TFAutoModel.from_pretrained(config.PRETRAINED_MODEL, from_pt=config.from_pt)
    output_layer = bert_model(input_layer) 
    pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[0])
    output = tf.keras.layers.Dense(30, activation='sigmoid')(pooled_output)
    classifier_model = tf.keras.Model(input_layer, output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=config.LR)
    classifier_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    classifier_model.summary()

    return classifier_model

In [94]:
classifier_model = build_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7feb720b3110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7feb720b3110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
global_average_pooling1d (Gl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 30)                23070     
Total params: 109,505,310
Trainable params: 109,505,310
Non-trainable params: 0
_________________________________________________________________


In [95]:
!nvidia-smi

Sun Jan 31 17:59:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    31W / 250W |   1419MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
classifier_model.fit(X_train, y_train, epochs=3, batch_size=8, validation_split=0.1)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/3


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


 417/3109 [===>..........................] - ETA: 24:20 - loss: 0.2052 - accuracy: 0.2865